In [ ]:
import os
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from itertools import product

# ----------------------------------
# Paths
# ----------------------------------
model_dir = "../models"
data_dir = "../data/labeled"

datasets = {
    "breast": pd.read_csv(f"{data_dir}/breast_labeled.csv"),
    "ovarian": pd.read_csv(f"{data_dir}/ovarian_labeled.csv"),
    "lung": pd.read_csv(f"{data_dir}/lung_labeled.csv")
}

models = {
    "breast": joblib.load(f"{model_dir}/breast_model.pkl"),
    "ovarian": joblib.load(f"{model_dir}/ovarian_model.pkl"),
    "lung": joblib.load(f"{model_dir}/lung_model.pkl")
}

# ----------------------------------
# Helper: align columns
# ----------------------------------
def align_genes(model, df):
    X = df.drop(columns=["status"])
    y = df["status"]
    model_genes = list(model.feature_names_in_)
    
    # Fill missing genes with 0
    for gene in model_genes:
        if gene not in X.columns:
            X[gene] = 0

    # Align order
    X = X[model_genes]
    return X, y

# ----------------------------------
# Cross-Disease Testing
# ----------------------------------
results = []

for train_name, test_name in product(models.keys(), repeat=2):
    model = models[train_name]
    test_df = datasets[test_name]

    X_test, y_test = align_genes(model, test_df)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_prob)

    print(f"\n🧬 {train_name.capitalize()}-Model → {test_name.capitalize()}-Data")
    print(f"   Accuracy: {acc:.4f} | ROC-AUC: {roc:.4f}")
    print(classification_report(y_test, y_pred, zero_division=0))

    results.append({
        "Model": train_name,
        "Tested_On": test_name,
        "Accuracy": acc,
        "ROC-AUC": roc
    })

# ----------------------------------
# Save results
# ----------------------------------
os.makedirs("../outputs/metrics", exist_ok=True)
pd.DataFrame(results).to_csv("../outputs/metrics/cross_disease_results.csv", index=False)
print("\n💾 Cross-disease results saved to ../outputs/metrics/cross_disease_results.csv")



🧬 Breast-Model → Breast-Data
   Accuracy: 1.0000 | ROC-AUC: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        43

    accuracy                           1.00        86
   macro avg       1.00      1.00      1.00        86
weighted avg       1.00      1.00      1.00        86


🧬 Breast-Model → Ovarian-Data
   Accuracy: 0.8413 | ROC-AUC: 0.8566
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.84      1.00      0.91        53

    accuracy                           0.84        63
   macro avg       0.42      0.50      0.46        63
weighted avg       0.71      0.84      0.77        63


🧬 Breast-Model → Lung-Data
   Accuracy: 0.9228 | ROC-AUC: 0.9732
              precision    recall  f1-score   support

           0       1.00      0.05      0.10        20
           1       0.92      1.00 

C:\Users\Admin\AppData\Local\Temp\ipykernel_11184\1227500249.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[gene] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_11184\1227500249.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[gene] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_11184\1227500249.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instea

In [1]:
# =======================================================
# 🌍 Cross-Cancer Biomarker Model Training
# Author: Sindhu S. Patil
# Description:
#   Builds universal models using common & union biomarkers
#   from breast, ovarian, and lung datasets
# =======================================================

import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import joblib

# --------------------------------------------------------
# ⚙️ Setup paths
# --------------------------------------------------------
DATA_DIR = "../data/labeled"
METRICS_DIR = "../outputs/metrics"
MODEL_DIR = "../models"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)

# --------------------------------------------------------
# 📂 Load top 100 gene lists
# --------------------------------------------------------
breast_top = pd.read_csv(f"{METRICS_DIR}/breast_top100_genes.csv")["Gene"].tolist()
ovarian_top = pd.read_csv(f"{METRICS_DIR}/ovarian_top100_genes.csv")["Gene"].tolist()
lung_top = pd.read_csv(f"{METRICS_DIR}/lung_top100_genes.csv")["Gene"].tolist()

# --------------------------------------------------------
# 🧬 Compute common and union genes
# --------------------------------------------------------
common_genes = sorted(list(set(breast_top) & set(ovarian_top) & set(lung_top)))
union_genes = sorted(list(set(breast_top) | set(ovarian_top) | set(lung_top)))

print(f"🧩 Common genes (intersection): {len(common_genes)}")
print(f"🌐 Union genes (combined): {len(union_genes)}")

# Save them
pd.Series(common_genes, name="Gene").to_csv(f"{METRICS_DIR}/common_genes.csv", index=False)
pd.Series(union_genes, name="Gene").to_csv(f"{METRICS_DIR}/union_genes.csv", index=False)

# --------------------------------------------------------
# 📊 Load datasets (breast + ovarian + lung)
# --------------------------------------------------------
breast = pd.read_csv(f"{DATA_DIR}/breast_labeled.csv")
ovarian = pd.read_csv(f"{DATA_DIR}/ovarian_labeled.csv")
lung = pd.read_csv(f"{DATA_DIR}/lung_labeled.csv")

# --------------------------------------------------------
# 🧩 Merge all datasets for universal model
# --------------------------------------------------------
combined = pd.concat([breast, ovarian, lung], ignore_index=True, sort=False)
print(f"\n📊 Combined dataset shape: {combined.shape}")

# --------------------------------------------------------
# ⚗️ Helper: Train and evaluate model
# --------------------------------------------------------
def train_model(df, genes, label_col="status", tag="common"):
    print(f"\n🚀 Training cross-cancer model using {tag} genes ({len(genes)} features)...")

    # Filter dataset
    df_filtered = df.dropna(subset=[label_col])
    available_genes = [g for g in genes if g in df_filtered.columns]

    X = df_filtered[available_genes].fillna(0)
    y = df_filtered[label_col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    model = RandomForestClassifier(
        n_estimators=400,
        random_state=42,
        class_weight="balanced",
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"✅ Accuracy: {acc:.4f}")
    print(f"✅ ROC-AUC: {auc:.4f}")
    print(classification_report(y_test, y_pred))

    # Extract feature importances
    feature_importances = pd.DataFrame({
        "Gene": available_genes,
        "Importance": model.feature_importances_
    }).sort_values(by="Importance", ascending=False)

    top_genes = feature_importances.head(50)
    print("\n🔥 Top 10 Biomarkers:")
    print(top_genes.head(10))

    # Save model + top genes
    model_path = f"{MODEL_DIR}/cross_cancer_{tag}_model.pkl"
    genes_path = f"{METRICS_DIR}/cross_cancer_{tag}_top50_genes.csv"

    joblib.dump(model, model_path)
    top_genes.to_csv(genes_path, index=False)

    print(f"\n💾 Model saved: {model_path}")
    print(f"💾 Top biomarkers saved: {genes_path}")

# --------------------------------------------------------
# 🧠 Train 2 Models:
# 1. Common Genes (intersection)
# 2. Union Genes (all unique)
# --------------------------------------------------------
if len(common_genes) > 0:
    train_model(combined, common_genes, tag="common")
else:
    print("⚠️ No common genes found across all datasets — skipping common model.")

train_model(combined, union_genes, tag="union")

print("\n🎯 Cross-Cancer Biomarker Training Complete!")


🧩 Common genes (intersection): 0
🌐 Union genes (combined): 293

📊 Combined dataset shape: (395, 22190)
⚠️ No common genes found across all datasets — skipping common model.

🚀 Training cross-cancer model using union genes (293 features)...
✅ Accuracy: 0.9367
✅ ROC-AUC: 0.9792
              precision    recall  f1-score   support

           0       0.81      0.87      0.84        15
           1       0.97      0.95      0.96        64

    accuracy                           0.94        79
   macro avg       0.89      0.91      0.90        79
weighted avg       0.94      0.94      0.94        79


🔥 Top 10 Biomarkers:
         Gene  Importance
1       ABCA8    0.039428
231    RASIP1    0.039389
71      FABP4    0.039087
19   ARHGEF10    0.029325
203     PCDH9    0.026549
50    CXorf57    0.026474
211     PLCE1    0.026003
119     KCNJ8    0.025350
65       EML1    0.023083
207    PECAM1    0.021121

💾 Model saved: ../models/cross_cancer_union_model.pkl
💾 Top biomarkers saved: ../output

In [2]:
# =======================================================
# 🧬 Cross-Cancer Similar Gene Model Training
# Author: Sindhu S. Patil
# =======================================================

import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import joblib

# --------------------------------------------------------
# ⚙️ Setup
# --------------------------------------------------------
DATA_DIR = "../data/labeled"
METRICS_DIR = "../outputs/metrics"
MODEL_DIR = "../models"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)

# --------------------------------------------------------
# 📂 Load top 100 gene lists
# --------------------------------------------------------
breast = pd.read_csv(f"{METRICS_DIR}/breast_top100_genes.csv")
ovarian = pd.read_csv(f"{METRICS_DIR}/ovarian_top100_genes.csv")
lung = pd.read_csv(f"{METRICS_DIR}/lung_top100_genes.csv")

breast_genes = set(breast["Gene"])
ovarian_genes = set(ovarian["Gene"])
lung_genes = set(lung["Gene"])

# --------------------------------------------------------
# 🧩 Find overlapping genes
# --------------------------------------------------------
common_all = breast_genes & ovarian_genes & lung_genes
pairwise_overlap = (breast_genes & ovarian_genes) | (breast_genes & lung_genes) | (ovarian_genes & lung_genes)
union_all = breast_genes | ovarian_genes | lung_genes

print(f"Common (all 3): {len(common_all)}")
print(f"Pairwise overlap (≥2): {len(pairwise_overlap)}")
print(f"Union (all unique): {len(union_all)}")

# Decide which set to use
if len(pairwise_overlap) > 0 and len(pairwise_overlap) <= 100:
    final_genes = sorted(list(pairwise_overlap))
    used_set = "pairwise_overlap"
elif len(common_all) > 0 and len(common_all) <= 100:
    final_genes = sorted(list(common_all))
    used_set = "common_all"
else:
    # fallback: top 100 from union
    print("⚠️ Using top 100 most important genes across all models (union fallback).")
    all_genes = pd.concat([
        breast.assign(Source="Breast"),
        ovarian.assign(Source="Ovarian"),
        lung.assign(Source="Lung")
    ])
    # Average importance across datasets
    avg_imp = (all_genes.groupby("Gene")["Importance"]
               .mean()
               .sort_values(ascending=False)
               .head(100))
    final_genes = avg_imp.index.tolist()
    used_set = "union_top100"

print(f"✅ Selected {len(final_genes)} genes from set: {used_set}")

# Save selected genes
pd.Series(final_genes, name="Gene").to_csv(f"{METRICS_DIR}/cross_selected_genes.csv", index=False)

# --------------------------------------------------------
# 📊 Load and combine datasets
# --------------------------------------------------------
df_breast = pd.read_csv(f"{DATA_DIR}/breast_labeled.csv")
df_ovarian = pd.read_csv(f"{DATA_DIR}/ovarian_labeled.csv")
df_lung = pd.read_csv(f"{DATA_DIR}/lung_labeled.csv")

combined = pd.concat([df_breast, df_ovarian, df_lung], ignore_index=True, sort=False)
print(f"📊 Combined shape: {combined.shape}")

# --------------------------------------------------------
# 🧠 Train model on selected genes
# --------------------------------------------------------
available_genes = [g for g in final_genes if g in combined.columns]
print(f"🧬 Using {len(available_genes)} genes found in data")

X = combined[available_genes].fillna(0)
y = combined["status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    class_weight="balanced",
    n_jobs=-1
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print(f"\n✅ Accuracy: {acc:.4f}")
print(f"✅ ROC-AUC: {auc:.4f}")
print(classification_report(y_test, y_pred))

# --------------------------------------------------------
# 💾 Save model and genes
# --------------------------------------------------------
model_path = f"{MODEL_DIR}/cross_overlap_{used_set}_model.pkl"
joblib.dump(model, model_path)
print(f"\n💾 Model saved to: {model_path}")

print("🎯 Cross-Cancer Overlap Model Training Complete!")


Common (all 3): 0
Pairwise overlap (≥2): 7
Union (all unique): 293
✅ Selected 7 genes from set: pairwise_overlap
📊 Combined shape: (395, 22190)
🧬 Using 7 genes found in data

✅ Accuracy: 0.9241
✅ ROC-AUC: 0.9724
              precision    recall  f1-score   support

           0       0.76      0.87      0.81        15
           1       0.97      0.94      0.95        64

    accuracy                           0.92        79
   macro avg       0.87      0.90      0.88        79
weighted avg       0.93      0.92      0.93        79


💾 Model saved to: ../models/cross_overlap_pairwise_overlap_model.pkl
🎯 Cross-Cancer Overlap Model Training Complete!


In [3]:
# =======================================================
# 🧪 Cross-Cancer Model Evaluation Script
# Author: Sindhu S. Patil
# Description:
#   Tests cross-cancer models (union + overlap) on
#   individual disease datasets: Breast, Ovarian, Lung.
# =======================================================

import os
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# --------------------------------------------------------
# ⚙️ Setup Paths
# --------------------------------------------------------
DATA_DIR = "../data/labeled"
MODEL_DIR = "../models"
OUTPUT_DIR = "../outputs/metrics"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------------------------------------
# 📦 Load Datasets
# --------------------------------------------------------
datasets = {
    "Breast": pd.read_csv(f"{DATA_DIR}/breast_labeled.csv"),
    "Ovarian": pd.read_csv(f"{DATA_DIR}/ovarian_labeled.csv"),
    "Lung": pd.read_csv(f"{DATA_DIR}/lung_labeled.csv"),
}

# --------------------------------------------------------
# 🧠 Load Models
# --------------------------------------------------------
models = {
    "Cross-Union": joblib.load(f"{MODEL_DIR}/cross_cancer_union_model.pkl"),
    "Cross-Overlap": joblib.load(f"{MODEL_DIR}/cross_overlap_pairwise_overlap_model.pkl")
}

# --------------------------------------------------------
# 🧩 Helper: Align Genes Between Model & Dataset
# --------------------------------------------------------
def align_genes(model, df):
    X = df.drop(columns=["status"])
    y = df["status"]

    model_genes = list(model.feature_names_in_)

    # Fill missing genes with 0, keep consistent order
    missing_genes = [g for g in model_genes if g not in X.columns]
    if missing_genes:
        X = pd.concat([X, pd.DataFrame(0, index=X.index, columns=missing_genes)], axis=1)
    X = X[model_genes]
    return X, y

# --------------------------------------------------------
# 🔍 Evaluate Models
# --------------------------------------------------------
results = []

for model_name, model in models.items():
    print(f"\n============================")
    print(f"🧠 Evaluating {model_name} Model")
    print(f"============================")

    for dataset_name, df in datasets.items():
        X, y = align_genes(model, df)

        y_pred = model.predict(X)
        y_prob = model.predict_proba(X)[:, 1]

        acc = accuracy_score(y, y_pred)
        auc = roc_auc_score(y, y_prob)

        print(f"\n📂 {model_name} → {dataset_name} Dataset")
        print(f"✅ Accuracy: {acc:.4f}")
        print(f"✅ ROC-AUC:  {auc:.4f}")
        print(classification_report(y, y_pred, zero_division=0))

        # Save confusion matrix
        cm = confusion_matrix(y, y_pred)
        disp = ConfusionMatrixDisplay(cm, display_labels=["Normal", "Cancer"])
        disp.plot(cmap="Blues")
        plt.title(f"{model_name} → {dataset_name} Confusion Matrix")
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/{model_name}_{dataset_name}_confusion.png")
        plt.close()

        results.append({
            "Model": model_name,
            "Tested_On": dataset_name,
            "Accuracy": acc,
            "ROC-AUC": auc
        })

# --------------------------------------------------------
# 💾 Save Summary
# --------------------------------------------------------
results_df = pd.DataFrame(results)
results_path = f"{OUTPUT_DIR}/cross_model_evaluation_summary.csv"
results_df.to_csv(results_path, index=False)

print("\n💾 Evaluation summary saved to:", results_path)
print("🎯 Cross-model testing complete!")



🧠 Evaluating Cross-Union Model

📂 Cross-Union → Breast Dataset
✅ Accuracy: 0.9767
✅ ROC-AUC:  0.9976
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        43
           1       0.98      0.98      0.98        43

    accuracy                           0.98        86
   macro avg       0.98      0.98      0.98        86
weighted avg       0.98      0.98      0.98        86


📂 Cross-Union → Ovarian Dataset
✅ Accuracy: 0.9841
✅ ROC-AUC:  1.0000
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       0.98      1.00      0.99        53

    accuracy                           0.98        63
   macro avg       0.99      0.95      0.97        63
weighted avg       0.98      0.98      0.98        63


📂 Cross-Union → Lung Dataset
✅ Accuracy: 0.9919
✅ ROC-AUC:  0.9977
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       